In [2]:
import pandas as pd
import numpy as np
import eli5

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [3]:
train_values = pd.read_csv('Nepal_Earthquake_train_values.csv')
train_labels = pd.read_csv('Nepal_Earthquake_train_labels.csv')

train_data = train_values.merge(train_labels,left_on = 'building_id',
                               right_on = 'building_id')

train_data.dtypes

building_id                                int64
geo_level_1_id                             int64
geo_level_2_id                             int64
geo_level_3_id                             int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_c

In [4]:
train_data = train_data.drop('building_id',axis = 1)

numeric_features = train_data.select_dtypes(include = ['int64',
                   'float64']).drop(['damage_grade'],axis = 1).columns

categorical_features = train_data.select_dtypes(include = 
                                                ['object']).columns

X = train_data.drop('damage_grade',axis = 1)
y = train_data['damage_grade']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.2)

In [5]:
numeric_transformer = Pipeline(steps = [('imputer',SimpleImputer(strategy = 'median')),
                                        ('scaler',StandardScaler())])

categorical_transformer = Pipeline(steps = [('imputer',SimpleImputer(strategy = 'constant',
                                                                    fill_value = 'missing')),
                                            ('one_hot', OneHotEncoder())])

preprocessor = ColumnTransformer(
                    transformers = [
                        ('num',numeric_transformer ,numeric_features),
                        ('cat',categorical_transformer ,categorical_features)
                    ])

pipe = Pipeline(steps = [('preprocessor',preprocessor),
                         ('classifier', 
                        LogisticRegression(class_weight = 'balanced',random_state = 0))])


model = pipe.fit(X_train,y_train)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


# Inspecting the quality of pipeline 

In [6]:
target_names = y_test.unique().astype(str)
y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred,target_names = target_names))

              precision    recall  f1-score   support

           2       0.45      0.61      0.52      5007
           3       0.64      0.63      0.63     29763
           1       0.49      0.44      0.46     17351

    accuracy                           0.57     52121
   macro avg       0.52      0.56      0.54     52121
weighted avg       0.57      0.57      0.57     52121



In [7]:
onehot_columns = list(pipe.named_steps['preprocessor'].named_transformers_['cat'].
                     named_steps['one_hot'].get_feature_names(input_features = 
                                                               categorical_features))


numeric_features_list = list(numeric_features)
numeric_features_list.extend(onehot_columns)

In [8]:
eli5.explain_weights(pipe.named_steps['classifier'],top = 50,
                    feature_names = numeric_features_list)

Explanation(estimator="LogisticRegression(C=1.0, class_weight='balanced', dual=False,\n                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,\n                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',\n                   random_state=0, solver='warn', tol=0.0001, verbose=0,\n                   warm_start=False)", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=1, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='plan_configuration_n', weight=0.7173909263167567, std=None, value=None), FeatureWeight(feature='other_floor_type_s', weight=0.37210252367113006, std=None, value=None), FeatureWeight(feature='position_s', weight=0.3534341398489303, std=None, value=None), FeatureWeight(feature='plan_configuration_a', weight=0.317989665178275, std=None, value=None), FeatureWeight(feature='foundation_type_i', weight=0.2754699268393418, std=None, value=None), FeatureWeight(feature='plan_configuration_o', weight=0.2535780582541876, std=None, value=None), FeatureWeight(feature='geo_level_1_id', weight=0.24706255903638685, std=None, value=None), FeatureWeight(feature='foundation_type_h', weight=0.23729596741389458, std=None, value=None), FeatureWeight(feature='ground_floor_type_v', weight=0.20551853849603216, std=None, value=None), FeatureWeight(feature='has_secondary_use_hotel', weight=0.1650793664593826, std=None, value=None), FeatureWeight(feature='has_superstructure_timber', weight=0.14653726726106547, std=None, value=None), FeatureWeight(feature='legal_ownership_status_a', weight=0.11610551595730732, std=None, value=None), FeatureWeight(feature='has_secondary_use_rental', weight=0.11492724672282935, std=None, value=None), FeatureWeight(feature='has_superstructure_rc_engineered', weight=0.11141212489446911, std=None, value=None), FeatureWeight(feature='has_secondary_use_other', weight=0.08263218075837482, std=None, value=None), FeatureWeight(feature='plan_configuration_c', weight=0.08262151349286205, std=None, value=None), FeatureWeight(feature='has_superstructure_other', weight=0.06246402070197528, std=None, value=None), FeatureWeight(feature='has_superstructure_rc_non_engineered', weight=0.05916926892732261, std=None, value=None), FeatureWeight(feature='roof_type_x', weight=0.058243715293799604, std=None, value=None), FeatureWeight(feature='has_secondary_use_agriculture', weight=0.054988162718652936, std=None, value=None), FeatureWeight(feature='geo_level_2_id', weight=0.05420847491703258, std=None, value=None)], neg=[FeatureWeight(feature='plan_configuration_f', weight=-1.1717422893463456, std=None, value=None), FeatureWeight(feature='foundation_type_r', weight=-0.6677283888346544, std=None, value=None), FeatureWeight(feature='position_o', weight=-0.5712855482239872, std=None, value=None), FeatureWeight(feature='legal_ownership_status_w', weight=-0.5639937594394091, std=None, value=None), FeatureWeight(feature='has_superstructure_mud_mortar_stone', weight=-0.5393406332744506, std=None, value=None), FeatureWeight(feature='<BIAS>', weight=-0.5039324573017073, std=None, value=None), FeatureWeight(feature='roof_type_q', weight=-0.485196892650954, std=None, value=None), FeatureWeight(feature='foundation_type_u', weight=-0.3779970670507539, std=None, value=None), FeatureWeight(feature='land_surface_condition_o', weight=-0.3412138482798441, std=None, value=None), FeatureWeight(feature='plan_configuration_m', weight=-0.33926909547444756, std=No